<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Copenhagen_Denmark_meteostat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
pip install meteostat

In [44]:
from meteostat import Hourly
from datetime import datetime
import pandas as pd
import warnings

# 1. Suppress the messy Pandas FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# --- CONFIGURATION ---
# We use the specific ID for Copenhagen Kastrup Airport (06180)
# instead of relying on coordinates which might pick a broken station.
STATION_ID = '06180'

START = datetime(2024, 1, 1)
END = datetime(2025, 1, 1)

print(f"Fetching data for Copenhagen Kastrup (ID: {STATION_ID})...")

try:
    # 2. Fetch by ID directly
    data = Hourly(STATION_ID, START, END)
    data = data.fetch()

    if not data.empty:
        df = data.reset_index()

        # Rename columns
        rename_map = {
            'time': 'Time',
            'temp': 'Temperature_C',
            'wspd': 'Wind_Speed_kmh',
            'rad': 'GHI_Wm2'
        }
        df = df.rename(columns=rename_map)

        # Wind Conversion (km/h -> m/s)
        if 'Wind_Speed_kmh' in df.columns:
            df['Wind_Speed_ms'] = df['Wind_Speed_kmh'] / 3.6

        # Date Formatting
        df['Date'] = df['Time'].dt.strftime('%Y-%m-%d')
        df['Hour'] = df['Time'].dt.strftime('%H:%M')

        # Select Columns
        cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms']

        # Check for Solar Radiation
        if 'GHI_Wm2' in df.columns and not df['GHI_Wm2'].isna().all():
            cols.append('GHI_Wm2')
            print(" > Success: Solar Radiation found!")
        else:
            print(" > Note: Solar Radiation is missing (common for airport stations).")

        df_final = df[[c for c in cols if c in df.columns]]

        print("-" * 30)
        print(df_final.head())
        print("-" * 30)

        filename = "Copenhagen_Kastrup_2024.csv"
        df_final.to_csv(filename, index=False)
        print(f"Success! Saved to {filename}")

    else:
        print(f"Error: No data found for Station {STATION_ID}. Meteostat might be updating.")

except Exception as e:
    print(f"An error occurred: {e}")

Fetching data for Copenhagen Kastrup (ID: 06180)...
 > Note: Solar Radiation is missing (common for airport stations).
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms
0  2024-01-01  00:00            5.4       4.111111
1  2024-01-01  01:00            5.4       5.111111
2  2024-01-01  02:00            5.6       5.694444
3  2024-01-01  03:00            5.5       5.111111
4  2024-01-01  04:00            5.6       4.111111
------------------------------
Success! Saved to Copenhagen_Kastrup_2024.csv
